## Setting up an optimisation with input constraints

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var3 = ContinuousVariable(name="variable3", lower_bound=0.0, upper_bound=100.0)
var4 = ContinuousVariable(name="variable4", lower_bound=0.0, upper_bound=100.0)
var5 = CategoricalVariableWithDescriptors(
    name="Cat_Var_Desc1",
    descriptor_names=["descriptor1", "descriptor2", "descriptor3"],
    categorical_levels=["cat_level1", "cat_level2", "cat_level3", "cat_level4"],
    categorical_descriptors=np.array(
        [
            ["0.1", "1", "60"],
            ["0.15", "2", "65"],
            ["0.25", "4", "75"],
            ["0.3", "6", "90"],
        ]
    ),
)
var6 = CategoricalVariableWithDescriptors(
    name="Cat_Var_Desc2",
    descriptor_names=["descriptor1", "descriptor2"],
    categorical_levels=["cat_level1", "cat_level2", "cat_level3"],
    categorical_descriptors=np.array([["0.1", "1", "60"], ["0.15", "2", "65"], ["0.25", "4", "75"]]),
)
var_list = VariablesList([var1, var2, var3, var4, var5, var6])

### Input constraints

Input constraints are used for both the sample generator and the acquisition functions. The examples below are a hypothetical example of how the constraints can be used with a set of continuous and categorical variables. The docstrings for the constraints comprehensively explains the arguments and keyword arguments.

By default in NEMO, all inequality constraints must be defined as 'greater than or equal to'.

Note: This aligns with how constraints are defined in the scipy package, but is the opposite when compared to the pymoo package

In [ ]:
from nemo_bo.opt.constraints import (LinearConstraint, NonLinearPowerConstraint, FunctionalConstraint, 
StoichiometricConstraint, MaxActiveFeaturesConstraint, CategoricalConstraint, ConstraintsList)

# The total of "variable1", "variable2", "variable3" must equal 100
constraint1 = LinearConstraint(constraint_type='eq',
                                    variables=var_list,
                                    features=["variable1", "variable2", "variable3"],
                                    coefficients=[1, 1, 1],
                                    rhs=100)

# (3 * ("variable1" ^ 2)) + "variable2" must greater than or equal to 40
constraint2 = NonLinearPowerConstraint(constraint_type='ineq',
                                    variables=var_list,
                                    features=["variable1", "variable2"],
                                    coefficients=[3, 1],
                                    powers=[2, 1],
                                    rhs=40)

# Define the function that calculates the left-hand-side of the constraint and pass it into the FunctionalConstraint class
def fun(x, **fun_kwargs):
    return (x[1] - np.sin(x[2]))

# The value calculated by the 'fun' method using "variable2" and "variable3" must be greater than or equal to 10
constraint3 = FunctionalConstraint(constraint_type='ineq',
                                    variables=var_list,
                                    features=["variable2", "variable3"],
                                    fun=fun,
                                    rhs=10,
                                    )

# The stoichiometric ratio of variable1 / variable 3 must be greater than or equal to 3
constraint4 = StoichiometricConstraint(constraint_type='ineq',
                                    variables=var_list,
                                    feature1="variable3",
                                    feature2="variable1",
                                    ratio=3)

# Out of "variable1", "variable2", "variable3", and "variable4", only 3 of them can be used (i.e. greater than zero) 
# for any experiment
constraint5 = MaxActiveFeaturesConstraint(variables=var_list,
                                    feature1=["variable1", "variable2", "variable3", "variable4"],
                                    max_active=3)

# For categorical features "Cat_Var_Desc1" and "Cat_Var_Desc2", "cat_level1" from "Cat_Var_Desc1" can not be used at 
# the same time as either "cat_level2" or "cat_level3" from "Cat_Var_Desc2"
constraint6 = CategoricalConstraint(variables=var_list,
                                    feature1="Cat_Var_Desc1",
                                    feature2="Cat_Var_Desc2",
                                    categorical_levels1=["cat_level1"]
                                    categorical_levels2=["cat_level2", "cat_level3"]
                                    max_active=3)

const_list = ConstraintsList([constraint1, constraint2, constraint3, constraint4, constraint5, constraint6])


In [ ]:
# Create the objective objects
obj1 = RegressionObjective(name="objective1", obj_max_bool=True, lower_bound=0.0, upper_bound=100.0)
obj_list = ObjectivesList([obj1])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling(num_new_points=2**20)

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance with the constraints
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler, constraints=const_list)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)